# Data Management Exercise using ISA Model and ISA API 

This exercise, in the form of an jupyter notebook, is meant to identify key elements about experimental design from an actual study narrative and build a digital archive from those. This is achieved by structuring experimental metadata following a principled approach, driven by experimental design concepts, by relying on a python application programming interface (API), the ISA-API. This ISA-API supports the [Investigation Study Assay Metadata model](http://isa-tools.org/) 

### Objectives
- Recognize and identify the key experimental design descriptors.
- Formalize the experimental design concepts using the ISA-API model. 
- Generate the study's metadata archive, corresponding to the experiment narrative, as an ISA format .

### Authors and Contact:
- http://isa-tools.org/team/
- [get in touch](mailto:isatools@googlegroups.com)


### References

- [Understanding ipython notebooks](https://github.com/danielskatz/repro-fdtd1d/blob/master/Notebook_Demonstration.ipynb), an jupyter notebook itself, refer to this document should you need any guidance on how to operate a notebook.

- [Notebook user interface](https://jupyter-notebook.readthedocs.io/en/stable/notebook.html#notebook-user-interface)

- [Documentation about the ISA-API](https://isatools.readthedocs.io), and more specifically:

    - [documentation about study-design driven creation of ISA content](https://isatools.readthedocs.io/en/latest/studydesigncreation.html)
    
    - [documentation of main ISA objects](https://isatools.readthedocs.io/en/latest/creation.html#)


# Case Study 1: A Toxicity Study in Rats

*Read the following experiment description **carefully**, paying **particular attention** to descriptors related to core concepts of experimental design.*


Experiment narrative:
-------------------------

   - *Male Fisher F344 rats* purchased from Charles River were treated with either of *three chemical compounds* commonly used painkillers, namely (aspirin, paracetamol and ibuprofen), at *two distinct dose levels* (low dose and high dose), delivered *per os*. Observations were made at *three different time points* (1 hour, 2 hours and 4 hours).
    
   - Equal numbers of animals (n=5) were allocated to each of the possible study groups, defined by a unique *compound, dose level and duration post exposure*  combination. Following sacrifice, performed by cervical dislocation preceded by anesthesia (ketamine and xylazine solution), *blood* and *liver* specimens (1 each per animal) were collected.
   
   - For liver samples, *total RNAs* was extracted; *transcription profiling* using *nucleotide sequencing* was performed using *paired-end libraries* on an *Illumina platform* using an HiSeq 2000 as *sequencing instrument*.
   
   - For blood samples, collected at sacrifice time and immediately placed in precooled 60 percent methanol ammonium bicarbonate buffer to extinguish cellular metabolism, *metabolites* were extracted in either *water-soluble fractions* or *lipophilic fractions*. *Metabolite profiling* using *mass spectrometry* was performed on the polar metabolite fraction only, using an *Agilent 6550 iFunnel Q-TOF Mass Sprectrometry instrument* platform. Each fraction was injected by flow injection analysis (FIA) twice and data were acquired in both ionization modes (i.e. positive mode and negative mode). Raw data files were saved in the native instrument format and later converted to [mzML](https://fairsharing.org/FAIRsharing.26dmba), the HUPO-PSI standard format for mass spectrometry.


## Structuring experimental description with ISA metadata model

To structure experimental information, we will use the models and objects defined by the ISA-API.

We therefore need to make python aware of such module. We do so with a series of ```import``` statements.

These are necessary to make the relevant functions available to our environment.

In [ ]:
# a library providing javascript widgets in the context of jupyter/ipython notebooks.
from ipywidgets import IntSlider

# the library providing ISA model and objects
from isatools.create.models import *
from isatools.model import *
from isatools.isatab import dump_tables_to_dataframes as dumpdf
from isatools.create.models import SampleAssayPlanEncoder
import isatools

# a library to providing support for json documents
import json

## Identification of variables, their levels and definition of the Treatment Plans

In the context of the experiment described in narrative, **can you identify the experiment independent variables and their associated levels**?

*(You may wish to write them down in a table where the first columns is the variable name, and the second columns its associated values)*

| Factor | Factor Values   |
|--------|-----------------|
|   f    | fv1,fv2,fv3     |

Once done, declare the corresponding ISA objects. 

  - To do so, one will have to create *a new object for each of the identified variables*, by relying on the ISA ```StudyFactor``` object, using the following syntax:

``` var1 = StudyFactor(name="chemical agent") ```
 
 - Augmenting the object declaration by performing some semantic markup. To do so, you may add an ontology term for such study factor, for instance using the [EBI Ontology Lookup Service](https://www.ebi.ac.uk/ols/) to find a relevant term from the ChEBI ontology, and build the factor in this way:
    
    *( **Important**: In this context, the tagging is for variable itself, not its values.)*
    
``` var1 = StudyFactor(name="chemical agent", 
                   factor_type=OntologyAnnotation(term="chemical entity",
                                                  term_source="ChEBI",
                                                  term_accession="http://purl.obolibrary.org/obo/CHEBI_24431"
                                                  )) ```

- Define the relevant ```StudyFactor```s below and see if you can find ontology terms that are relevant to annotate them (for instance, you may wish to try and find terms from the [Experimental Factor Ontology](https://www.ebi.ac.uk/ols/ontologies/efo)):


In [ ]:
### Alter or augment the statement below to match the experimental description:

var1 = StudyFactor(name="chemical agent")


Next, we can instantiate the ```TreatmentFactory``` class and start adding the ISA ```StudyFactor``` and associated factor levels (i.e. the values defined by the experimentalist and that the factor will assume over the course of experiment execution).

``` treatment_factory = TreatmentFactory(factors=[ ... here list the variables...]) ```

**Important**: Pay attention to the nature (type of data structure) of the ```factors``` attribute for the ```TreatmentFactory``` class. It is a list (aslo known as an array in other languages), a data structure which can hold more than one element.


In [ ]:
### Alter or augment the statement below to match the experimental description:

treatment_factory = TreatmentFactory(factors=[var1])


and then for each factor, you will have to add all their associated levels in the following way:

```python
treatment_factory.add_factor_value( ... factor ..., { ... list of strings with the names of the factor values... }
```

Note: invoke the ```add_factor_value``` function as many times as needed to add to ```treatment_factor``` instance.

In [ ]:
treatment_factory.add_factor_value(var1, {'value 1', 'value 2'})

### Computing the Number of Unique Treatment Groups/Study Groups: 


The experiment plan follows a *full factorial design*, which means that all possible treatments, that is the  combinations of factor values, are used.
We can obtain the different treatments by relying on a utility method that, given various sets of factors and their levels, computes the cartesian products over those sets to produce the actual set of possible treatments.


In [ ]:
all_treatments = treatment_factory.compute_full_factorial_design()

How many treatment groups have been computed?  Get this number by issueing the following command:

In [ ]:
print('Number of study groups (treatment groups): {}'.format(len(all_treatments)))

Does the number resulting from executing the command correspond to what you had in mind ? If not, why?

We can now build a treatment plan, or ```TreatmentSequence```, by adding all the treatments computed but the factorial design method.

In [ ]:
treatment_sequence = TreatmentSequence(ranked_treatments=all_treatments)

Are study subjects exposed to a single intervention or do they receive multiple consecutive interventions?
  


### View the treatment plan information as a JSON document
You can now output a summary of the treatment plan that you created with the following command:

In [ ]:
report = make_summary_from_treatment_sequence(treatment_sequence)
report

Is the treatment plan report in agreement with the experimental design?

### Study group size

The following code builds a slider (relying on the ```ipywidgets``` library) for you to set the group size, please select the appropriate number according to the experiment description, where group size is the number of subjects per each combination of factor values or treatment group:

In [ ]:

group_size = IntSlider(value=1, min=0, max=10, step=1, description='Group size:', disabled=False, continuous_update=False, orientation='horizontal', readout=True, readout_format='d')
group_size


The group size value you chose (and that is going to be used in the next section) can be viewed with the following command:

In [ ]:
group_size.value

 ### Study group characteristics:
 
 Given the current setting, what can you say about the study design ? balanced or unbalanced?

## Sample collection and assay plans

Given the study group size selected above, which represents the number of subjects allocated to each of the treatment groups, we will be building two things: 1) the sample collection plan and 2) assay plans.

In [ ]:
plan = SampleAssayPlan(group_size=group_size.value)

Let's now build the sample collection plan using a python ```dictionary```. It should contain key:value pairs, where  specimen or 'sample type' terms are used as dictionary keys and the number of samples collected for each 'sample type' over the course of the study as associated dictionary values. Below is the code snippet that you should complete:

```python
sample_collection_plan = { "sample type 1": 2, "sample type 2": 1 }
```


In [ ]:
### Alter or augment the statement below to match the experimental description:

sample_collection_plan = { "sample type 1": 2, "sample type 2": 1 }


Next, the following code will take the sample_collection_plan object that you built and include all the details in the sample_collection_plan object:

In [ ]:
for sample_type in sample_collection_plan:    
    plan.add_sample_type(sample_type)
    plan.add_sample_plan_record(sample_type,sample_collection_plan[sample_type])


### View the sample plan information as a JSON document

This section is meant to show how to write key study design descriptors in a compact document serialized in JSON format. 
Why is this relevant? How would you use this feature? List 3 possible uses.

In [ ]:
print(json.dumps(plan, cls=SampleAssayPlanEncoder, sort_keys=True, indent=4, separators=(',', ': ')))

## Create an ISA experimental description based on the study design and the sampling plan:

In the following section, the task is to build ISA objects relying on the study design information we built above.


Let's first create an ```Investigation``` object to hold all the information about the experiment with the following instruction, which also assigns an identifier for the investigation.

In [ ]:
isa_investigation = Investigation(identifier='inv-dtp-exercise')

Now, let's create a study object using the sample and assay plan as well as the instance of the ```TreatmentSequence``` object we've built before. For this, we will need an object of the ```IsaModelObjectFactory``` class provided by the ISA-API:

In [ ]:
isa_object_factory = IsaModelObjectFactory(plan,treatment_sequence)
isa_study = isa_object_factory.create_study_from_plan()

Now, we can link the ISA study object to the ISA investigation object we created earlier:

In [ ]:
isa_investigation.studies = [isa_study]



Next, we need to set a name for the ISA study table file. Considering the ISA specifications, the convention to follow is that the filename must start with ```s_``` and have a ```.txt``` extension and could have any string in between such that it makes a valid file name. Thus the patter to follow is: ```s_some-string.txt```.

In [ ]:
### HERE YOUR ANSWER

isa_study.filename = 's_(complete here a filename for the study file).txt'

Let's now check the study sample table:

In [ ]:
dataframes = dumpdf(isa_investigation)
try:
    sample_table = next(iter(dataframes.values()))
    display(sample_table)
except StopIteration:
    print("Need more details to print table")


In [ ]:
try:
    print('Total rows generated: {}'.format(len(sample_table)))
except NameError:
    print("Need more details to print result")

### Study description and study design type


Can you set the study description (or abstract) relying in this code snippet?

```python
isa_study.description = "... here the text of the description..."
```


In [ ]:
### HERE YOUR ANSWER

isa_study.description = "... here the text of the description..."


Next, we would like to specify the type of study design (and we can set multiple values if necessary). 

For this, we will build ontology annotations, as we did for the ```StudyFactor`` objects:

```python
descriptor_1 = OntologyAnnotation(
                  term="... here the label of the term...", 
                  term_source="... here the name of the ontology the term comes from...", 
                  term_accession="... here the URL of the term ...")
```

To determine some of the study design descriptors, consider the following questions and use the Ontology Lookup Service to find relevant terms:

- is the experiment following an 'intervention design' or an 'observation design'?
- is the design 'factorial' or a 'randomized block' design?
- is the design 'full' or 'fractional'?

In [ ]:
### HERE DEFINE YOUR DESCRIPTORS

After you defined the descriptors, you can append them to isa_study.design_descriptors list as follows:

```python
isa_study.design_descriptors.append(descriptor_1)
```


In [ ]:
### HERE APPEND ALL THE DESCRIPTORS YOU DEFINED ABOVE

## Assay and Data Acquisition Plans:

From the experiment narrative, identify the *response variables* (a.k.a.[dependent variables](https://goo.gl/tfGNQe)).  Map those to ISA compatible terms and definitions.

The ISA model defines an ```*ISA Assay Type*``` by a pair of descriptors:  a *type of measurement*  and the *type of technology* used to obtain readings from the response variables.

A series of configuration files that define the vetted values for Measurement Type and Technology type. We have extracted a few of those in the table below:


| Measurement Type | Technology type   |
|:-|----------------------:|
|   transcription profiling    | DNA microarray     |
|   transcription profiling    | nucleic acid sequencing     |
|   transcription factor binding site identification    | DNA microarray     |
|   transcription factor binding site identification    | nucleic acid sequencing     |
|   metabolite profiling    | mass spectrometry    |
|   metabolite profiling    | nmr spectroscopy     |
|   targeted gene survey    | nucleic acid sequencing     |
|   histopathology    | microscopy imaging     |
|   phenotyping    | multispectral imaging     |



Given this set, define the assay types matching the experiment narrative.

The way to define an ``` ISA assay type``` using the ISA-API is as follows:

``` assay_type_1= AssayType(measurement_type='...here a supported measurement type...', technology_type='...here a supported technology type...') ```

Use as many statements as necessary:

In [ ]:
### HERE YOUR ANSWER
assay_type_1= AssayType(measurement_type='...here a supported measurement type...', 
                        technology_type='...here a supported technology type...')


Let's now define a Python ```set``` [(a kind of data structure)](https://realpython.com/python-sets/) for the assay types:

In [ ]:
assay_types = set()

You can add the types you just defined above to the ```set``` as follows:
    
``` assay_types.add(assay_type_1) ```

Add all your assay types to the ```ISA assay_types``` set below, using as many statements as necessary:

In [ ]:
### HERE YOUR ANSWER
assay_types.add(assay_type_1)


Let's now visualise the assay types that you created:

In [ ]:
for x in assay_types:
        print(x.measurement_type.term," using ", x.technology_type.term)


## Assay Specific Descriptors:

 - Each data acquisition modality comes with its own set of parameters related to design of experiment, and aiming to account for technical variability. These parameters can be set as to capture the specifics of the underlying experimental workflow, which be can accommodated by the ISA model.

 - This section of the exercise aims to extract the technical specifics for each of the data acquisition experimental workflows as declared in the experimental narrative and population the relevant objects from the ISA model.


### Dealing with Next Generation Sequencing (NGS) Data Acquisition Plan

Select in the following slider the number of technical replicates in the Next Generation Sequence (NGS) data acquisition plan:

In [ ]:
ngs_technical_replicates = IntSlider(value=1, min=1, max=5, step=1, description='Technical repeats:', disabled=False, continuous_update=False, orientation='horizontal', readout=True, readout_format='d')
ngs_technical_replicates



Let's now identify the instrument and include it in a set:

In [ ]:
### HERE YOUR ANSWER
instrument = '... here include the instrument name...'

In [ ]:
sequencing_instruments = set()
sequencing_instruments.add(instrument) 

 - How many different libraries were used for NGS data acquisition? What were their types? If you don't know what a sequencing library is, the following [link](https://emea.illumina.com/science/technology/next-generation-sequencing/paired-end-vs-single-read-sequencing.html) will provide background information.

 - Set the value in the slider below (and you can change the default value so that you don't need to change the slider every time you re-run the cell):

In [ ]:
ngs_distinct_libraries = IntSlider(value=1, min=1, max=5, step=1, description='Distinct libraries:', disabled=False, continuous_update=False, orientation='horizontal', readout=True, readout_format='d')
ngs_distinct_libraries


The value you selected was:

In [ ]:
ngs_distinct_libraries.value

- The NGS parameters that you selected in the previous steps are known as ```'topology modifiers'``` in ISA speak . This name refers to the fact that they affect the shape of the experimental graph being considered, where the graph has material or data entities as nodes and processes as edges.

- The following code defines a ```DNASeqAssayTopologyModifiers``` object and adds it to the AssayType object defined earlier on:

In [ ]:
top_mods_seq = DNASeqAssayTopologyModifiers(
    technical_replicates=ngs_technical_replicates.value, 
    instruments=sequencing_instruments,
    distinct_libraries=ngs_distinct_libraries.value)

assay_type_1.topology_modifiers = top_mods_seq

print(assay_type_1.topology_modifiers)

Now that we have all the information about the NGS assay type, we can add it to the plan:

In [ ]:
plan.add_assay_type(assay_type_1)

Finally, for the NGS, we need to identify to what sample type it is applied. Fill in the NGS sample type in the following variable:

In [ ]:
### HERE YOUR ANSWER

ngs_sample_type = '... here include the sample type for which NGS is applied...'

The following code will associate the NGS assay type to the relevant sample type (as per your definition above):

In [ ]:
for sample_type in plan.sample_types:
    if sample_type.value.term == ngs_sample_type:
        print('adding assay for sample_type '+sample_type.value.term)
        plan.add_assay_plan_record(sample_type.value.term, assay_type_1)
        assay_plan = next(iter(plan.assay_plan))
    elif sample_type.value.term != ngs_sample_type:
        print('doing nothing for '+sample_type.value.term)
        

Let's now check the assay plan to see if it matches the experiment narrative:

In [ ]:
try:
    print('Added assay plan: {0} -> {1}/{2}'.format(assay_plan[0].value.term, assay_plan[1].measurement_type.term, assay_plan[1].technology_type.term))
    if len(top_mods_seq.instruments) > 0:
        print('Instruments: {}'.format(list(top_mods_seq.instruments)))
except NameError:
    print("Need more details to print assay plan")

In [ ]:
print(json.dumps(plan, cls=SampleAssayPlanEncoder, sort_keys=True, indent=4, separators=(',', ': ')))

### Dealing with Mass Spectrometry Data Acquisition Plan

Moving on, we'll now capture the information about second assay type, related to mass spectrometry.
We want to identify, when relevant:
- the chromatography instruments
- the mass spectrometry instrument
- the injection mode
- the acquisition modes

Given the definition of the following sets, add the relevant elements as described in the experiment narrative:

In [ ]:
chromatography_instruments = set()
ms_instruments = set()
injection_modes = set()
acquisition_modes = set()


In [ ]:
### HERE YOUR ANSWER

How many technical replicates are used for the mass spectrometry assay? Set the appropriate value below:


In [ ]:
### HERE YOUR ANSWER 
ms_tech_rep = 0

Given all this information, we are in a position to define an object of the class ```MSAssayTopologyModifiers``` and assign it to the corresponding assay type:

In [ ]:
try:
    top_mods_ms = MSAssayTopologyModifiers(
          technical_replicates=ms_tech_rep, 
          injection_modes=injection_modes, 
          acquisition_modes=acquisition_modes, 
          instruments=ms_instruments, 
          chromatography_instruments=chromatography_instruments)

    assay_type_2.topology_modifiers = top_mods_ms
except NameError:
    print("There was an assay type left undefined")

Let's see all the settings to check they are as expected:

In [ ]:
if len(top_mods_ms.chromatography_instruments) > 0:
    print('Chromatography instruments: {}'.format(list(top_mods_ms.chromatography_instruments)))
else:
    print('no chromatography used or no information supplied')

if len(top_mods_ms.instruments) > 0:
    print('Data acquisition instruments: {}'.format(list(top_mods_ms.instruments)))    
if len(top_mods_ms.injection_modes) > 0:
    print('Injection modes: {}'.format(list(top_mods_ms.injection_modes)))
if len(top_mods_ms.acquisition_modes) > 0:
    print('Acquisition modes: {}'.format(list(top_mods_ms.acquisition_modes)))


Then, we add the mass spectrometry assay to the plan:

In [ ]:
try:
    plan.add_assay_type(assay_type_2)
    plan.add_assay_plan_record("blood", assay_type_2)

    assay_plan = next(iter(plan.assay_plan))
except NameError:
    print("There was an assay type left undefined")

And let's see the json representation of the SampleAssayPlan:

In [ ]:
print(json.dumps(plan, 
                 cls=SampleAssayPlanEncoder, 
                 sort_keys=True, 
                 indent=4, 
                 separators=(',', ': ')))

As the last step, we generate assay tables from the information on the Assay Plans and visualise the output files.

In [ ]:
isa_investigation.studies = [isa_object_factory.create_assays_from_plan()]

for assay in isa_investigation.studies[-1].assays:
    print('Assay generated: {0}, {1} samples, {2} processes, {3} data files'
          .format(assay.filename, 
                  len(assay.samples), 
                  len(assay.process_sequence), 
                  len(assay.data_files)))
dataframes = dumpdf(isa_investigation)

In [ ]:
import pandas as pd
table = pd.DataFrame()
try:
    table = dataframes[next(iter(dataframes.keys()))]
except StopIteration:
    print("Need more information to print the table")
    
display(table)

In [ ]:
table = pd.DataFrame()
try:
    table = dataframes['a_blood_ms_FIA_positive_assay.txt']
except KeyError:
    print("Need more details to print the assay table")
display(table)   

In [ ]:
table = pd.DataFrame()
try:
    table = dataframes['a_blood_ms_FIA_negative_assay.txt']
except KeyError:
    print("Need more details to print the assay table")
display(table)   

In [ ]:
table = pd.DataFrame()
try:
    table = dataframes['a_liver_dnaseq_Illumina HiSeq 2000_assay.txt']
except KeyError:
    print("Need more details to print the assay table")
display(table)    